# DuckDB tutorial

[DuckDB](https://duckdb.org/) is a column-oriented database ... (description)

In this tutorial, we will introduce the basics of DuckDB, as well as more advanced queries. 
We will also demonstrate its use on a dedicated dataset and compare it to a well-known row oriented database, sqlite. This will show the advantages and drawbacks of both systems depending on the workload.

## Setup

First, create a Python virtual environment. The following shell command will create a virtual environment in the current directory named 'duckdb':


``
python -m venv duckdb
``

We can activate the environment using either on UNIX-like systems:

``
source duckdb/bin/activate
``

or on Microsoft Windows:

``
.\duckdb\Scripts\activate
``

N.B.: On Windows, this might require to [change the scripts execution policy](https://learn.microsoft.com/en-us/powershell/module/microsoft.powershell.security/set-executionpolicy?view=powershell-7.5) on the host machine. In powershell, one can change it using the following cmdlet. This will allow the system to run scripts for the current process (i.e. terminal session).

``
Set-ExecutionPolicy -ExecutionPolicy AllSigned -Scope Process
``

Once the virtual environment is activated, we can start downloading the required libraries.

``
pip install -r requirements.txt
``

We are now ready to go.



In [20]:
# If this cell executes correctly, you have all the dependencies needed for this tutorial.

import time
import duckdb
import pandas as pd
import sqlite3  # Directly present in python, no need to pip install it

# Hello, DuckDB!
We will simply test our DuckDB installation by printing a traditional Hello World. This basic script will create a `.db` file which is able to persist data even after the connexion is closed.

In [21]:
# This is a simple test to see if duckdb works
# Will create the file if it doesn't exist
test_con = duckdb.connect('test.db')

query = "CREATE TABLE IF NOT EXISTS hello AS SELECT 'Hello, world!'"

test_con.execute(query)

fetch = "SELECT * FROM hello"

result = test_con.execute(fetch).fetchall()

print(result)

test_con.close()

[('Hello, world!',)]


# Dataset
We will now fetch a dataset that best suits our needs. 

In [22]:
# Load a small subset of NYC Taxi data from a parquet file
url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet'
df = pd.read_parquet(url)
df = df.head(10000)
print(f"Dataset shape: {df.shape}")


Dataset shape: (10000, 19)


# Dump the dataset to a .db file
We will now dump a subpart of the dataset into dedicated .db files, each of them respectively managed by sqlite and duckdb. 

In [23]:
# Connect to a local DuckDB file (creates it if it doesn't exist)
duckdb_con = duckdb.connect('taxi_data.db')

# Register the Pandas DataFrame as a temporary view
duckdb_con.register('temp_view', df)

# Dump the data: Create a persistent table from the view
duckdb_con.execute("CREATE TABLE IF NOT EXISTS trips AS SELECT * FROM temp_view")

# Verify (optional): Query a sample
print(duckdb_con.sql("SELECT * FROM trips LIMIT 5").df())

# Close the connection (data is now persisted in the file)
duckdb_con.close()

   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         2  2023-01-01 00:32:10   2023-01-01 00:40:36              1.0   
1         2  2023-01-01 00:55:08   2023-01-01 01:01:27              1.0   
2         2  2023-01-01 00:25:04   2023-01-01 00:37:49              1.0   
3         1  2023-01-01 00:03:48   2023-01-01 00:13:25              0.0   
4         2  2023-01-01 00:10:29   2023-01-01 00:21:19              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           0.97         1.0                  N           161           141   
1           1.10         1.0                  N            43           237   
2           2.51         1.0                  N            48           238   
3           1.90         1.0                  N           138             7   
4           1.43         1.0                  N           107            79   

   payment_type  fare_amount  extra  mta_tax  tip_amount  tolls_amount  \


In [24]:
# Connect to a local SQLite file (creates it if it doesn't exist)
sqlite_con = sqlite3.connect('taxi_data.sqlite')

# Dump the data: Write the DataFrame to a table (replaces if exists)
df.to_sql('taxi_data', sqlite_con, if_exists='replace')

# Testing by querying the database
print(pd.read_sql_query("SELECT * FROM taxi_data LIMIT 5", sqlite_con))
sqlite_con.close()

   index  VendorID tpep_pickup_datetime tpep_dropoff_datetime  \
0      0         2  2023-01-01 00:32:10   2023-01-01 00:40:36   
1      1         2  2023-01-01 00:55:08   2023-01-01 01:01:27   
2      2         2  2023-01-01 00:25:04   2023-01-01 00:37:49   
3      3         1  2023-01-01 00:03:48   2023-01-01 00:13:25   
4      4         2  2023-01-01 00:10:29   2023-01-01 00:21:19   

   passenger_count  trip_distance  RatecodeID store_and_fwd_flag  \
0              1.0           0.97         1.0                  N   
1              1.0           1.10         1.0                  N   
2              1.0           2.51         1.0                  N   
3              0.0           1.90         1.0                  N   
4              1.0           1.43         1.0                  N   

   PULocationID  DOLocationID  payment_type  fare_amount  extra  mta_tax  \
0           161           141             2          9.3   1.00      0.5   
1            43           237             1     